<a href="https://colab.research.google.com/github/jnises/llmog/blob/finetuning/training/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is based on the [unsloth](https://unsloth.ai/) gemma 3 finetune notebook.

This can be run on a T4.

In [ ]:
# upgrading datasets seems to solve an issue when loading datasets: `NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported.`
# this seems to require us to restart the notebook sometimes, so we put it first
!pip install --upgrade datasets

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = False, # we are training a 1B model, so we don't need to quantize much
    load_in_8bit = True,
    full_finetuning = False,
)

In [ ]:
# We now add LoRA adapters so we only need to update a small amount of parameters

model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [ ]:
# load the dataset from huggingface

import datasets
dataset = datasets.load_dataset('jnises/llmog-conversations', split = 'train')

In [ ]:
# We now use standardize_data_formats to try converting datasets to the correct format for finetuning purposes

from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

In [ ]:
# We now have to apply the chat template for Gemma-3 onto the conversations, and save it to text.
# We remove the <bos> token using removeprefix('<bos>') since we're finetuning.
# The Processor will add this token before training and the model expects only one.

def formatting_prompts_func(examples):
  convos = examples["conversations"]
  texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
  return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 30, # uncomment this and comment the num_train_epochs line to do a quicker run
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
        dataset_num_proc=2,
    ),
)

In [ ]:
# We also use Unsloth's train_on_completions method to only train on the assistant outputs and ignore the loss on the user's inputs.
# This helps increase accuracy of finetunes.

from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
# Let's train the model.
# To resume a training run, set trainer.train(resume_from_checkpoint = True)

trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
# Test the trained model.
# According to the Gemma-3 team, the recommended settings for inference are temperature = 1.0, top_p = 0.95, top_k = 64

if False:
  from unsloth.chat_templates import get_chat_template
  tokenizer = get_chat_template(
      tokenizer,
      chat_template = "gemma-3",
  )
  messages = [
      {
                  "role": "system",
                  'content': [{'type': 'text', 'text':
                              '''You are a developer log analyzer.
  Given a sequence of log lines. Rate only the last line. Use the prior lines only for context.
  If a prior line looks unrelated to the last one, disregard it.
  Rate the last line by how interesting you think it is for diagnosing an issue with the system.
  Output EXACTLY in this format:
  ```
  Very brief single-sentence analysis on a single line
  SCORE: 0-100
  ```

  Do NOT include any code examples, snippets, or additional explanations.
  Keep responses strictly limited to the analysis and score.
  Do NOT include any additional framing such as ````.
  Do NOT start the analysis with "The last line" or similar redundant information.

  Score guide:
  Low (0-30): Routine/minor info
  Medium (31-70): Noteworthy/important
  High (71-100): Critical/security issues
  '''
  }]},
      {
      "role": "user",
      "content":
      [{
          "type" : "text",
          "text" : '''[2025-02-25 08:07:33] [INFO] Transaction started - xid=a7f392e1
  [2025-02-25 08:07:33] [WARNING] Slow database query detected - duration=1530ms - query="SELECT * FROM orders JOIN order_items WHERE customer_id = ?"
  [2025-02-25 08:07:33] [INFO] Transaction committed - xid=a7f392e1
  ''',
      }]
  }]
  text = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt = True, # Must add for generation
      # this seems to be needed for the 1B model
      tokenize=False,
  )
  outputs = model.generate(
      **tokenizer([text], return_tensors = "pt").to("cuda"),
      max_new_tokens = 64, # Increase for longer outputs!
      # Recommended Gemma-3 settings!
      temperature = 1.0, top_p = 0.95, top_k = 64,
  )
  tokenizer.batch_decode(outputs)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model_name = 'gemma-3-1b-llmog'
#model_name = 'gemma-3-1b-llmog-test'
HF_REPO = f"jnises/{model_name}"

In [ ]:
# model.save_pretrained(model_name)  # Local saving
# tokenizer.save_pretrained(model_name)

In [ ]:
# model.push_to_hub(HF_REPO, token = HF_TOKEN) # Online saving
# tokenizer.push_to_hub(HF_REPO, token = HF_TOKEN) # Online saving

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
# if True: # Change to True to save finetune!
#     model.save_pretrained_merged(model_name, tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if True: # Change to True to upload finetune
    model.push_to_hub_merged(
        HF_REPO, tokenizer,
        token = HF_TOKEN,
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
# if True: # Change to True to save to GGUF
#     model.save_pretrained_gguf(
#         # need to be just the model name here, a full path results in confusing error messages
#         model_name,
#         quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
#     )

In [ ]:
# import shutil
# import os
# if model_path.exists():
#   shutil.rmtree(model_path)
# gguf_name = f'{model_name}.Q8_0.gguf'
# gguf_path = model_base_path / gguf_name
# if gguf_path.exists():
#   os.unlink(gguf_path)
# shutil.copytree(model_name, model_path, dirs_exist_ok=True, ignore=shutil.ignore_patterns('.cache'))
# shutil.copy(gguf_name, gguf_path)

In [ ]:
# %ls drive/MyDrive/colab/llmog/model_output/gemma-3-llmog

In [ ]:
#%cp -r gemma-3-llmog* drive/MyDrive/colab/

In [ ]:

# %ls

In [ ]:
# %rm gemma-3-llmog.Q8_0.gguf

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
# if True: # Change to True to upload GGUF
#     model.push_to_hub_gguf(
#         #model_name,
#         HF_REPO,
#         quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
#         #repo_id = HF_REPO,
#         token = HF_TOKEN,
#     )

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
